In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest, chi2
import pickle
import random

In [2]:
# Load the dataset
data = pd.read_csv('diab_clean_version.csv')

In [3]:
data_copy = data.copy()

In [4]:
data.drop(['Unnamed: 0'], inplace = True , axis = 1)

In [5]:
#assigning 0 if less than 30 else 1
data['readmitted'] = data['readmitted'].apply(lambda x: 0 if x=='<30' else 1)

In [6]:
#saving readmitted 0 and value in file
data.to_csv('diab_clean_version.csv',index = False)

In [7]:
# Prepare the data
X = data.drop('readmitted', axis=1) # Feature matrix
y = data['readmitted'] # Target variable
X = pd.get_dummies(X) # One-hot encode categorical features
X = X.fillna(0) # Replace missing values with 0
X_new = SelectKBest(chi2, k=10).fit_transform(X, y)

In [8]:
# balance the classes using RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_new, y)

In [9]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [10]:
# tune the hyperparameters using GridSearchCV
param_grid = {'n_estimators': [100, 200, 300],
              'max_depth': [10, 20, 30],
              'min_samples_split': [2, 5, 10]}
clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

In [11]:
# create and train the random forest classifier with the best hyperparameters
model = RandomForestClassifier(random_state=42, **best_params)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=30, n_estimators=300, random_state=42)

In [12]:
# evaluate the model
y_pred = model.predict(X_test)

In [13]:
# calculate the accuracy, precision, recall, and f1-score
baseline_model_accuracy = accuracy_score(y_test, y_pred)
baseline_model_report = classification_report(y_test, y_pred)
print("Accuracy:", baseline_model_accuracy)
print("Classification Report:")
print(baseline_model_report)

Accuracy: 0.9527518321597931
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     17435
           1       0.99      0.91      0.95     17360

    accuracy                           0.95     34795
   macro avg       0.96      0.95      0.95     34795
weighted avg       0.96      0.95      0.95     34795



In [14]:
# Number of users among which we will split the data
n_users = 5

# Shuffle the data randomly
data = data.sample(frac=1)

# Split the data into n_users subsets each of same size
users_data = np.array_split(data, n_users)

In [15]:
users_data[1].shape

(19611, 21)

In [16]:
# Serialize and save the model
with open('initial_model.pkl', 'wb') as f:
    pickle.dump(model, f)
    
# Share the initial model with all users
for i in range(n_users):
    with open('initial_model.pkl', 'rb') as f:
        model_data = pickle.load(f)

In [17]:
#This code is training each user locally on their subset and returning accuracy and Classification-report
def train_model(model, local_data):
    # Load the initial ML model from a file
    with open('initial_model.pkl', 'rb') as f:
        model = pickle.load(f)
    
    X_train_i = local_data.drop('readmitted', axis=1) # Feature matrix
    y_train_i = local_data['readmitted'] # Target variable
    X_train_i = pd.get_dummies(X_train_i) # One-hot encode categorical features
    X_train_i = X_train_i.fillna(0) # Replace missing values with 0
    X_new = SelectKBest(chi2, k=10).fit_transform(X_train_i, y_train_i)
    
    
    # balance the classes using RandomOverSampler
    oversampler = RandomOverSampler(sampling_strategy='minority')
    X_resampled, y_resampled = oversampler.fit_resample(X_new, y_train_i)

        
   
    # split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.2, random_state=42)
    
    model.fit(X_train, y_train)
    
    
    
    y_pred = model.predict(X_test)
    accuracy_i = accuracy_score(y_test, y_pred)
    report_i =  classification_report(y_test, y_pred)
    return accuracy_i, report_i

In [18]:
# List to store locally trained model accuracy
trained_models = []
# List to store locally trained model classification-report
report_data =[]

# Loop over each user
for i in range(n_users):
    # Train the received model on local data
    local_model, report = train_model(model, users_data[i])
    
    # Add accuracy and classification-report to the list
    trained_models.append(local_model)
    report_data.append(report)

In [19]:
#printing accuracy of all the users
for i in range(n_users):
    print(f"accuracy of user{i} is {trained_models[i]}")

accuracy of user0 is 0.961688031110471
accuracy of user1 is 0.9640856198822009
accuracy of user2 is 0.9671118770644836
accuracy of user3 is 0.9685976484083739
accuracy of user4 is 0.9623292595255212


In [20]:
#printing classification-report of all the users
for i in range(n_users):
    print(report_data[i])

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      3435
           1       1.00      0.93      0.96      3508

    accuracy                           0.96      6943
   macro avg       0.96      0.96      0.96      6943
weighted avg       0.96      0.96      0.96      6943

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      3429
           1       1.00      0.93      0.96      3532

    accuracy                           0.96      6961
   macro avg       0.97      0.96      0.96      6961
weighted avg       0.97      0.96      0.96      6961

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3469
           1       1.00      0.94      0.97      3494

    accuracy                           0.97      6963
   macro avg       0.97      0.97      0.97      6963
weighted avg       0.97      0.97      0.97      6963

              preci

In [21]:
# This function is calculating avg of all the locally trained models and return as global model
def average_models(models):
    global_model = None
    num_models = len(models)
    
    # Compute the sum of all models
    for model in models:
        if global_model is None:
            global_model = model
        else:
            global_model += model
    
    # Divide the sum by the number of models to get the average
    global_model /= num_models
    
    return global_model

In [22]:
global_model = average_models(trained_models)

In [23]:
#accuracy of global model
global_model

0.9647624871982101

In [24]:
#Difference between baseline global model and Merged global model
diff_base_merge = baseline_model_accuracy - global_model
abs(diff_base_merge)

0.012010655038417029

In [25]:
#Difference between each baseline model with each locally trained model

for i in range(n_users):
    diff_base_local = abs(baseline_model_accuracy - trained_models[i])
    print(f"accuracy of user{i} : {diff_base_local}")

accuracy of user0 : 0.008936198950677876
accuracy of user1 : 0.011333787722407784
accuracy of user2 : 0.014360044904690561
accuracy of user3 : 0.015845816248580835
accuracy of user4 : 0.009577427365728086


## Make user[1] adversary by poisoning its dataset

In [26]:
#This function is used to poison the dataset:
def poison(data, fraction):
    #total_data = data.shape[0]
    user2_indices = range(0, 19610)
    modified_data = int(len(user2_indices) * fraction)
    indices = np.random.choice(user2_indices, modified_data, replace= False)
    data.loc[indices, 'readmitted'] = data.loc[indices, 'readmitted'].apply(lambda x: 0 if x== 1 else 1)
    #data.loc[indices, 'readmitted'] = data.loc[indices, 'readmitted'].apply(lambda x: '>30' if x== '<30' else '<30') 
    #data.to_csv("check.csv",index = False)
    return data

In [27]:
#Resetting the index of the user, whose data we are poisoning
users_data[1] = users_data[1].reset_index(drop=True)
adversarial_data = poison(users_data[1] , 0.4) #40% data poison
#adversarial_data = poison(users_data[1] , 0.1) #10% data poison


In [28]:
# List to store locally trained models accuracy
poisoned_trained_models = []
# List to store locally trained models classification-report
poisoned_report_data = []

# Loop over each participating user
for i in range(n_users):
    if i != 1:
        # Train the received model on local data
        local_model,poisoned_report = train_model(model, users_data[i])
    else:
        local_model, poisoned_report = train_model(model, adversarial_data)
        
    
    
   # Add accuracy and classification-report to the list
    poisoned_trained_models.append(local_model)
    poisoned_report_data.append(poisoned_report)

In [29]:
#printing accuracy of all the user, here user[1] accuracy drop, so it is possible that it is adversarial user:
for i in range(n_users):
    print(f"accuracy of user{i} is {poisoned_trained_models[i]}")

accuracy of user0 is 0.9576551922799943
accuracy of user1 is 0.5526548672566372
accuracy of user2 is 0.966393795777682
accuracy of user3 is 0.968741038141669
accuracy of user4 is 0.9629043853342919


In [30]:
#printing classification-report of all the user, here user[1] clssification is very poor:
for i in range(n_users):
    print(f"user{i} classification report:")
    print(poisoned_report_data[i])

user0 classification report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      3435
           1       0.99      0.92      0.96      3508

    accuracy                           0.96      6943
   macro avg       0.96      0.96      0.96      6943
weighted avg       0.96      0.96      0.96      6943

user1 classification report:
              precision    recall  f1-score   support

           0       0.54      0.56      0.55      2225
           1       0.56      0.54      0.55      2295

    accuracy                           0.55      4520
   macro avg       0.55      0.55      0.55      4520
weighted avg       0.55      0.55      0.55      4520

user2 classification report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3469
           1       1.00      0.94      0.97      3494

    accuracy                           0.97      6963
   macro avg       0.97      0.97      0.

In [31]:
poisoned_global_model = average_models(poisoned_trained_models)

In [32]:
#average accuracy of global model:
#Note: here accuracy decreased and one of the reason coud be adversarial user

print(f"global accuracy after data poisioning :{poisoned_global_model}")

global accuracy after data poisioning :0.8816698557580548


In [33]:
#Difference between baseline global model and Poisoned global model
diff_base_poisoned_merge = baseline_model_accuracy - poisoned_global_model
abs(diff_base_poisoned_merge)

0.0710819764017383

In [34]:
#Difference between each baseline model with each locally trained model after poisoning

for i in range(n_users):
    diff_base_poison_local = abs(baseline_model_accuracy - poisoned_trained_models[i])
    print(f"accuracy difference of user{i} with baseline model : {diff_base_poison_local}")
#Here you can notice that user[1] performing different from everyone. So we will remove it.

accuracy difference of user0 with baseline model : 0.004903360120201206
accuracy difference of user1 with baseline model : 0.40009696490315594
accuracy difference of user2 with baseline model : 0.013641963617888941
accuracy difference of user3 with baseline model : 0.01598920598187592
accuracy difference of user4 with baseline model : 0.010152553174498813


##   Remove Adversarial user and check performance

In [35]:
# List to store the accuracy of locally trained models
without_adversarial_trained_models = []
# List to store the classification-report of locally trained models
without_adversarial_report = []

# Loop over each participating user: we have included all the user except user[1]
for i in range(n_users):
    if i != 1:
        # Train the received model on local data
        local_model, adverse_report = train_model(model, users_data[i])
        #local_model, adverse_report = train_model(model, adversarial_data)
        
    
    
    # Add the trained model to the list of locally trained models
    without_adversarial_trained_models.append(local_model)
    without_adversarial_report.append(adverse_report)

In [36]:
#printing the accuracy of all the user except user[1]
for i in range(n_users):
    if i != 1:
        print(f"accuracy of user{i} : {without_adversarial_trained_models[i]}")

accuracy of user0 : 0.9619760910269336
accuracy of user2 : 0.9666810282924027
accuracy of user3 : 0.9680240894751936
accuracy of user4 : 0.9617541337167506


In [37]:
#finding the global average without the user which we suspect of being adversarial: in our case that user is user[1]
without_adversarial_global_model = average_models(without_adversarial_trained_models)

In [38]:
#notice that after removing user[1] our accuracy has improved as it was before poisoning 

print(f"accuracy of global model after removing adversary user: {without_adversarial_global_model}")

accuracy of global model after removing adversary user: 0.9640822867076428


In [39]:
#Difference between baseline global model and adverse user removed global model
diff_base_without_adversarialUser_merge = baseline_model_accuracy - without_adversarial_global_model
abs(diff_base_without_adversarialUser_merge)

0.011330454547849755

In [40]:
#Difference between each baseline model with each locally trained model except adversarial user
for i in range(n_users):
    if i != 1: 
        diff_base_poison_local = abs(baseline_model_accuracy - poisoned_trained_models[i])
        print(f"accuracy of user{i} : {diff_base_poison_local}")

accuracy of user0 : 0.004903360120201206
accuracy of user2 : 0.013641963617888941
accuracy of user3 : 0.01598920598187592
accuracy of user4 : 0.010152553174498813


In [41]:
data_copy.to_csv('diab_clean_version.csv',index = False) 